In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
import torch
import math
import re
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from tokenizers import ByteLevelBPETokenizer
from tqdm import tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\satvm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv("poetry.csv")

In [3]:
pd.set_option('display.max_colwidth', None)
df.tail()

,Unnamed: 0,Title,Poem,Poet,Tags
13849,13,\r\r\n 1-800-FEAR\r\r\n,\r\r\nWe'd like to talk with you about fear they said so\r\r\r\nmany people live in fear these days they drove up\r\r\r\nall four of them in a small car nice boy they said\r\r\r\nbeautiful dogs they said so friendly the man ahead\r\r\r\nof the woman the other two waiting in the drive I\r\r\r\nwas outside digging up the garden no one home I said\r\r\r\nwhat are you selling anyway I'm not interested I\r\r\r\nsaid well you have a nice day they said here's our\r\r\r\ncard there's a phone number you can call anytime\r\r\r\nany other houses down this road anyone else live\r\r\r\nhere we'd like to talk to them about living in fear\r\r\n,Jody Gladding,"Living,Social Commentaries,Popular Culture"
13850,14,\r\r\n The Death of Atahuallpa\r\r\n,\r\r\n\r\r\n,William Jay Smith,NaN
13851,15,\r\r\n Poet's Wish\r\r\n,\r\r\n\r\r\n,William Jay Smith,NaN
13852,0,\r\r\n 0\r\r\n,"\r\r\n Philosophic\r\r\nin its complex, ovoid emptiness,\r\r\na skillful pundit coined it as a sort\r\r\n of stopgap doorstop for those\r\r\n quaint equations Romans never\r\r\ndreamt of. In form completely clever\r\r\nand discrete—a mirror come unsilvered, loose watch face without the works, a hollowed globe from tip to toe\r\r\nunbroken, it evades the grappling\r\r\nhooks of mass, tilts the thin rim of no thing, remains embryonic sum, non-cogito.\r\r\n",Hailey Leithauser,"Arts & Sciences,Philosophy"
13853,1,\r\r\n !\r\r\n,"\r\r\nDear Writers, I’m compiling the first in what I hope is a series of publications I’m calling artists among artists. The theme for issue 1 is “Faggot Dinosaur.” I hope to hear from you! Thank you and best wishes.",Wendy Videlock,"Relationships,Gay, Lesbian, Queer,Arts & Sciences,Poetry & Poets,Social Commentaries,Gender & Sexuality"


In [4]:
df.isna().sum()

Unnamed: 0      0
Title           0
Poem            0
Poet            0
Tags          955
dtype: int64

In [5]:
df = df.drop(['Unnamed: 0', 'Tags', 'Poet', 'Title'], axis=1)

In [6]:
# df['Title'] = df['Title'].str.replace('\r', '')
df['Poem'] = df['Poem'].str.replace('\r', '')
# df['Title'] = df['Title'].str.replace('\n', '')
df['Poem'] = df['Poem'].str.strip('\n')

In [7]:
df.tail()

,Poem
13849,We'd like to talk with you about fear they said so\nmany people live in fear these days they drove up\nall four of them in a small car nice boy they said\nbeautiful dogs they said so friendly the man ahead\nof the woman the other two waiting in the drive I\nwas outside digging up the garden no one home I said\nwhat are you selling anyway I'm not interested I\nsaid well you have a nice day they said here's our\ncard there's a phone number you can call anytime\nany other houses down this road anyone else live\nhere we'd like to talk to them about living in fear
13850,
13851,
13852,"Philosophic\nin its complex, ovoid emptiness,\na skillful pundit coined it as a sort\n of stopgap doorstop for those\n quaint equations Romans never\ndreamt of. In form completely clever\nand discrete—a mirror come unsilvered, loose watch face without the works, a hollowed globe from tip to toe\nunbroken, it evades the grappling\nhooks of mass, tilts the thin rim of no thing, remains embryonic sum, non-cogito."
13853,"Dear Writers, I’m compiling the first in what I hope is a series of publications I’m calling artists among artists. The theme for issue 1 is “Faggot Dinosaur.” I hope to hear from you! Thank you and best wishes."


In [8]:
# Lower case everything
# df['Title'] = df['Title'].str.lower()
df['Poem'] = df['Poem'].str.lower()

# Remove apostrophes and join the parts
# df['Title'] = df['Title'].str.replace("'", "", regex=False).str.replace("’", "", regex=False)
df['Poem'] = df['Poem'].str.replace("'", "", regex=False).str.replace("’", "", regex=False)

In [9]:
# Uncomment only when including encoder, until then let it be commented off
# Filter out rows where both 'Title' and 'Poem' have no alphabetic characters

# df = df[df.apply(lambda x: any(c.isalpha() for c in x['Title']) and
#                             any(c.isalpha() for c in x['Poem']), axis=1)]

# Filter out rows with no value (its not NaN its '' in this dataset)
df = df[df.apply(lambda x: any(c.isalpha() for c in x['Poem']), axis=1)]

In [10]:
# Make newline a special token
# df['Title'] = 'START ' + df['Title'] + ' END'
df['Poem'] = 'START ' + df['Poem'] + ' END'

df['Poem'] = df['Poem'].apply(lambda x: re.sub(r'\n', ' N ', x))

<ipython-input-10-643c636f129a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Poem'] = 'START ' + df['Poem'] + ' END'
<ipython-input-10-643c636f129a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Poem'] = df['Poem'].apply(lambda x: re.sub(r'\n', ' N ', x))


In [11]:
def remove_numeric_tokens(text):
    tokens = word_tokenize(text)
    # Filter tokens to exclude any that contain digits
    filtered_tokens = [token for token in tokens if not re.search(r'\d', token)]
    processed_text = ' '.join(filtered_tokens)
    return processed_text

df['Poem'] = df['Poem'].apply(remove_numeric_tokens)

In [12]:
df.tail()

,Poem
13835,"START dear writers , im compiling the first in what i hope is a series of publications im calling artists among artists . the theme for issue is “ faggot dinosaur. ” i hope to hear from you ! thank you and best wishes . END"
13848,"START the wise men will unlearn your name . N above your head no star will flame . N one weary sound will be the same— N the hoarse roar of the gale . N the shadows fall from your tired eyes N as your lone bedside candle dies , N for here the calendar breeds nights N till stores of candles fail . N what prompts this melancholy key ? N a long familiar melody . N it sounds again . so let it be . N let it sound from this night . N let it sound in my hour of death— N as gratefulness of eyes and lips N for that which sometimes makes us lift N our gaze to the far sky . N you glare in silence at the wall . N your stocking gapes : no gifts at all . N its clear that you are now too old N to trust in good saint nick ; N that its too late for miracles . N —but suddenly , lifting your eyes N to heavens light , you realize : N your life is a sheer gift . END"
13849,START wed like to talk with you about fear they said so N many people live in fear these days they drove up N all four of them in a small car nice boy they said N beautiful dogs they said so friendly the man ahead N of the woman the other two waiting in the drive i N was outside digging up the garden no one home i said N what are you selling anyway im not interested i N said well you have a nice day they said heres our N card theres a phone number you can call anytime N any other houses down this road anyone else live N here wed like to talk to them about living in fear END
13852,"START philosophic N in its complex , ovoid emptiness , N a skillful pundit coined it as a sort N of stopgap doorstop for those N quaint equations romans never N dreamt of . in form completely clever N and discrete—a mirror come unsilvered , loose watch face without the works , a hollowed globe from tip to toe N unbroken , it evades the grappling N hooks of mass , tilts the thin rim of no thing , remains embryonic sum , non-cogito . END"
13853,"START dear writers , im compiling the first in what i hope is a series of publications im calling artists among artists . the theme for issue is “ faggot dinosaur. ” i hope to hear from you ! thank you and best wishes . END"


In [13]:
df.isna().sum()

Poem    0
dtype: int64

In [14]:
df.size

13751

In [15]:
tokenizer = ByteLevelBPETokenizer()

# Saving poems to a file (required for tokenizer training)
poem_file = "poems.txt"
with open(poem_file, "w", encoding='utf-8') as file:  # Specify UTF-8 encoding
    file.write('\n'.join(df['Poem']))

# Train the tokenizer on the poems
tokenizer.train(files=[poem_file], vocab_size=30000, min_frequency=1, special_tokens=["START", "END", "N", "UNK", "PAD"])

In [17]:
# Tokenize poems and store the token IDs
df['Tokenized_Poem'] = df['Poem'].apply(lambda poem: tokenizer.encode(poem).ids)
df['Token_Length'] = df['Tokenized_Poem'].apply(len)

# Find the longest lengths and sort them
sorted_lengths = sorted(df['Token_Length'], reverse=True)
top_15_lengths = sorted_lengths[:15]
print("Top 15 longest tokenized poem lengths:", top_15_lengths)

max_length = 10000

# Ensure END token is known or defined
END_token_id = tokenizer.token_to_id("END")

# Function to trim poems and append END token
def trim_and_append_end(poem_tokens):
    if len(poem_tokens) > max_length:
        return poem_tokens[:max_length] + [END_token_id]
    return poem_tokens

# Apply trimming and END token appending
df['Tokenized_Poem'] = df['Tokenized_Poem'].apply(trim_and_append_end)

# Update the 'Token_Length' to reflect the post-operation lengths
df['Token_Length'] = df['Tokenized_Poem'].apply(len)

# Appoint longest_length as the longest poem length after all operations
longest_length = max(df['Token_Length'])
print("Longest length after operations:", longest_length)

Top 15 longest tokenized poem lengths: [22321, 14606, 13912, 12385, 11251, 10661, 10216, 9995, 9930, 9822, 9761, 9622, 8799, 8577, 8125]
Longest length after operations: 10001


In [18]:
vocab_size = tokenizer.get_vocab_size()  # Number of unique words in the vocabulary
embedding_dim = 128
batch_size = 1 # Increase batch size if resources allow as it bring stabilization, 1 is very noisy
learning_rate = 0.01 # changed lr because maybe the embedding dim is too low and lr is too high so gradient is just bouncing around and not learning much
pad_index = 1
heads = 4
epochs = 1
device = "cuda"

In [19]:
pad_token = tokenizer.token_to_id("PAD")
padded_poems = torch.tensor([poem + [pad_token] * (longest_length - len(poem)) for poem in df['Tokenized_Poem']])

# Creating PyTorch DataLoader
dataset = TensorDataset(padded_poems)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Verify shapes (optional)
print(f"Shape of padded inputs tensor: {padded_poems.shape}")

Shape of padded inputs tensor: torch.Size([13751, 10001])


In [20]:
# for i in data_loader:
#     print(i[0].shape)
#     break

In [21]:
pad_token

4

In [22]:
class WordEmbedder(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

    def forward(self, x):
        return self.embeddings(x)

In [23]:
# Now that we have successfully trained our word embedding layer, time to make our positional matrix, which we will make using the formula mentioned in the paper
def positional_encoding(seq_len, embedding_dim, device=device):
    pe = torch.zeros(seq_len, embedding_dim, device=device)
    # Compute the positional encoding values
    position = torch.arange(0, seq_len, dtype=torch.float, device=device).unsqueeze(1)
    # Adjust div_term calculation to handle odd embedding_dim
    div_term_exp = torch.arange(0, embedding_dim, 2).float() * -(np.log(10000.0) / embedding_dim)
    div_term = torch.exp(div_term_exp).to(device)
    pe[:, 0::2] = torch.sin(position * div_term)
    if embedding_dim % 2 == 0:  # Check if embedding_dim is even
        pe[:, 1::2] = torch.cos(position * div_term)  # For even embedding_dim
    else:
        # Adjust the last cosine computation for odd embedding_dim
        pe[:, 1::2] = torch.cos(position * div_term)[:,:-1]  # Exclude the last column for odd embedding_dim
    return pe

In [24]:
encoder_pos_encoding = positional_encoding(longest_length, embedding_dim)
encoder_pos_encoding.shape

torch.Size([10001, 128])

In [25]:
# Time to make the Multi-head Self Attention block
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, heads, embedding_dim):
        super(MultiHeadSelfAttention, self).__init__()
        self.heads = heads

        # 3 Linear Layers for Q, K and V
        self.w_q = nn.Linear(embedding_dim, embedding_dim)
        self.w_k = nn.Linear(embedding_dim, embedding_dim)
        self.w_v = nn.Linear(embedding_dim, embedding_dim)

        # Since the feature or embedding dimension is typically the last dimension
        self.softmax = nn.Softmax(dim=-1)

        # Last Linear layer for the attention
        self.w_a = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, embedding_vector):
        batch_size, seq_len, embedding_dim = embedding_vector.size()
        # Compute Q, K, and V
        Q = self.w_q(embedding_vector)
        K = self.w_k(embedding_vector)
        V = self.w_v(embedding_vector)

        # Seperate into heads
        head_dim = embedding_dim // self.heads
        Q = Q.view(batch_size, self.heads, seq_len, head_dim)
        K = K.view(batch_size, self.heads, seq_len, head_dim)
        V = V.view(batch_size, self.heads, seq_len, head_dim)

        # Calculate attention
        attention = torch.matmul(self.softmax(torch.matmul(Q, K.transpose(-1, -2)) / torch.sqrt(torch.tensor(embedding_dim))), V)

        # Concatenating the attention heads (Transposing for correct concatenation)
        attention = attention.transpose(1, 2).reshape(batch_size, seq_len, embedding_dim)
        output = self.w_a(attention)
        return output

In [26]:
# Time to make the Masked Multi-head Self Attention block
class MaskedMultiHeadSelfAttention(nn.Module):
    def __init__(self, heads, embedding_dim):
        super(MaskedMultiHeadSelfAttention, self).__init__()
        self.heads = heads

        # 3 Linear Layers for Q, K and V
        self.w_q = nn.Linear(embedding_dim, embedding_dim)
        self.w_k = nn.Linear(embedding_dim, embedding_dim)
        self.w_v = nn.Linear(embedding_dim, embedding_dim)

        # Since the feature or embedding dimension is typically the last dimension
        self.softmax = nn.Softmax(dim=-1)

        # Last Linear layer for the attention
        self.w_a = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, embedding_vector):
        batch_size, seq_len, embedding_dim = embedding_vector.size()
        # Compute Q, K, and V
        Q = self.w_q(embedding_vector)
        K = self.w_k(embedding_vector)
        V = self.w_v(embedding_vector)

        # Seperate into heads
        head_dim = embedding_dim // self.heads
        Q = Q.view(batch_size, self.heads, seq_len, head_dim)
        K = K.view(batch_size, self.heads, seq_len, head_dim)
        V = V.view(batch_size, self.heads, seq_len, head_dim)

        # Create a mask for masking the attention score
        mask = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool), diagonal=1).unsqueeze(0).unsqueeze(0).expand(batch_size, self.heads, -1, -1).to(device)
        value_to_fill = float('-inf')

        # Calculate attention (including mask)
        attention = torch.matmul(self.softmax(torch.matmul(Q, K.transpose(-1, -2)).masked_fill(mask, value_to_fill) / torch.sqrt(torch.tensor(embedding_dim))), V)

        # Concatenating the attention heads (Transposing for correct concatenation)
        attention = attention.transpose(1, 2).reshape(batch_size, seq_len, embedding_dim)
        output = self.w_a(attention)
        return output

In [8]:
x = torch.randint(1, 10, size=(2, 3, 4, 5))
x

tensor([[[[6, 6, 3, 6, 2],
          [5, 9, 7, 9, 3],
          [8, 8, 5, 7, 2],
          [4, 9, 1, 5, 2]],

         [[9, 8, 1, 7, 1],
          [2, 7, 1, 2, 3],
          [8, 2, 8, 1, 1],
          [4, 3, 1, 1, 9]],

         [[7, 7, 9, 1, 4],
          [1, 2, 3, 1, 5],
          [5, 6, 8, 4, 3],
          [7, 8, 8, 8, 3]]],


        [[[8, 6, 6, 6, 3],
          [6, 3, 7, 3, 8],
          [8, 2, 6, 9, 9],
          [2, 4, 8, 1, 3]],

         [[9, 4, 1, 6, 8],
          [7, 2, 3, 2, 5],
          [3, 4, 4, 1, 3],
          [8, 8, 5, 3, 7]],

         [[4, 2, 3, 7, 7],
          [9, 2, 1, 1, 7],
          [4, 9, 3, 1, 8],
          [2, 3, 5, 6, 5]]]])

In [12]:
x.transpose(1, 2).reshape(2, 4, 15)

tensor([[[6, 6, 3, 6, 2, 9, 8, 1, 7, 1, 7, 7, 9, 1, 4],
         [5, 9, 7, 9, 3, 2, 7, 1, 2, 3, 1, 2, 3, 1, 5],
         [8, 8, 5, 7, 2, 8, 2, 8, 1, 1, 5, 6, 8, 4, 3],
         [4, 9, 1, 5, 2, 4, 3, 1, 1, 9, 7, 8, 8, 8, 3]],

        [[8, 6, 6, 6, 3, 9, 4, 1, 6, 8, 4, 2, 3, 7, 7],
         [6, 3, 7, 3, 8, 7, 2, 3, 2, 5, 9, 2, 1, 1, 7],
         [8, 2, 6, 9, 9, 3, 4, 4, 1, 3, 4, 9, 3, 1, 8],
         [2, 4, 8, 1, 3, 8, 8, 5, 3, 7, 2, 3, 5, 6, 5]]])

In [27]:
class AddNorm(nn.Module):
    def __init__(self, n_features):
        super(AddNorm, self).__init__()
        # Layer Norm will normalize the last dimension of the matrix
        self.norm = nn.LayerNorm(n_features)

    def forward(self, original, modified):
        return self.norm(original + modified)

In [28]:
class FeedForward(nn.Module):
    def __init__(self, embedding_dim):
        super(FeedForward, self).__init__()
        # Normally nn.Linear(embedding_dim, embedding_dim * 4) for expressiveness, we will change it if we have the resources to do so
        self.lr1 = nn.Linear(embedding_dim, embedding_dim)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        out = self.lr2(x)
        return out

In [29]:
# Time to build the Decoder
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, batch_size, heads):
        super(Decoder, self).__init__()
        self.word_embedder = WordEmbedder(vocab_size, embedding_dim)
        self.positional_encoding = positional_encoding(longest_length, embedding_dim)
        self.masked_attention = MaskedMultiHeadSelfAttention(heads, embedding_dim)
        self.add_norm1 = AddNorm(embedding_dim)
        self.attention = MultiHeadSelfAttention(heads, embedding_dim)
        self.add_norm2 = AddNorm(embedding_dim)
        self.feed_forward = FeedForward(embedding_dim)
        self.add_norm3 = AddNorm(embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        x = self.word_embedder(x)
        x = x + self.positional_encoding[:x.size(1), :].unsqueeze(0).expand(x.size(0), -1, -1)
        x = self.add_norm1(x, self.masked_attention(x))
        x = self.add_norm2(x, self.attention(x))
        x = self.add_norm3(x, self.feed_forward(x))
        logits = self.linear(x)
        return F.softmax(logits, dim=-1)

In [30]:
model = Decoder(vocab_size, embedding_dim, batch_size, heads).to(device)

In [31]:
# Initialize our optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [32]:
def train_epoch(data_loader, model, optimizer, loss_fn, device, pad_token):
    model.train()
    total_loss = 0.0

    # Wrap your data loader with tqdm for a progress bar
    progress_bar = tqdm(data_loader, desc="Training", leave=True)

    for batch in progress_bar:
        torch.cuda.empty_cache()
        poems = batch[0].to(device)  # Load poems to the device

        # Find the max length in this batch to minimize computation
        max_length = poems.ne(pad_token).sum(1).max().item()  # Assuming padding token ID is pad_token
        poems = poems[:, :max_length]  # Trim to max length

        optimizer.zero_grad()

        # Prepare input and target sequences
        input_sequences = poems[:, :-1]  # All tokens except the last
        target_sequences = poems[:, 1:]  # All tokens except the first

        # Forward pass using teacher forcing
        logits = model(input_sequences)

        # Calculate loss
        loss = loss_fn(logits.transpose(1, 2), target_sequences)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Update the progress bar with the current batch loss
        progress_bar.set_postfix(loss=loss.item())

    average_loss = total_loss / len(data_loader)
    return average_loss

for epoch in range(epochs):
    print(f"Starting Epoch {epoch + 1}/{epochs}")
    epoch_loss = train_epoch(data_loader, model, optimizer, loss_fn, device, pad_token)
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")

Starting Epoch 1/1


Training: 100%|██████████| 13751/13751 [15:36<00:00, 14.68it/s, loss=10.3]

Epoch 1, Loss: 10.2361


In [33]:
model_path = "model_weights.pth"
torch.save(model.state_dict(), model_path)

In [34]:
model.load_state_dict(torch.load("model_weights.pth"))
model.eval()

Decoder(
  (word_embedder): WordEmbedder(
    (embeddings): Embedding(30000, 128)
  )
  (masked_attention): MaskedMultiHeadSelfAttention(
    (w_q): Linear(in_features=128, out_features=128, bias=True)
    (w_k): Linear(in_features=128, out_features=128, bias=True)
    (w_v): Linear(in_features=128, out_features=128, bias=True)
    (softmax): Softmax(dim=-1)
    (w_a): Linear(in_features=128, out_features=128, bias=True)
  )
  (add_norm1): AddNorm(
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (attention): MultiHeadSelfAttention(
    (w_q): Linear(in_features=128, out_features=128, bias=True)
    (w_k): Linear(in_features=128, out_features=128, bias=True)
    (w_v): Linear(in_features=128, out_features=128, bias=True)
    (softmax): Softmax(dim=-1)
    (w_a): Linear(in_features=128, out_features=128, bias=True)
  )
  (add_norm2): AddNorm(
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (feed_forward): FeedForward(
    (lr1): Linear(in_f

In [35]:
def generate_poem(model, start_sequence, tokenizer=tokenizer, max_length=100):
    model.eval()  # Set the model to evaluation mode.
    tokens = tokenizer.encode(start_sequence)  # Encode the start sequence to tokens directly using the tokenizer.

    # Initialize the sequence with the token IDs of the start sequence
    generated_ids = tokens.ids

    for _ in range(max_length - len(generated_ids)):
        input_seq = torch.tensor([generated_ids], dtype=torch.long).to(device)
        with torch.no_grad():
            output_logits = model(input_seq)

        # Predict the next token ID (not index).
        next_token_id = output_logits[:, -1, :].argmax(-1).item()
        generated_ids.append(next_token_id)  # Append the ID of the next token.

        # Stop if the end token is generated.
        end_token_id = tokenizer.token_to_id("END")
        if next_token_id == end_token_id:
            break

    # Decode the token IDs back to text using the tokenizer.
    generated_text = tokenizer.decode(generated_ids)

    return generated_text

In [38]:
start_sequence = input("Please enter the starting words for your poem: ")
poem = generate_poem(model, start_sequence)
print(poem)

Please enter the starting words for your poem: the ball
the ball                                                                                                  
